<a href="https://colab.research.google.com/github/Narendra-Sompalle/GenAIProjects/blob/main/HybridSearch_Chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
! pip install --upgrade --quiet langchain_community langchain langchain-google-genai pypdf chromadb rank_bm25

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [40]:
documents=[
    "I like this phone",
    "I love this place",
    "you am from india"
]

In [41]:
vector=TfidfTransformer()

In [42]:
# First, convert the documents into a matrix of token counts
count_vectorizer = CountVectorizer()
document_term_matrix = count_vectorizer.fit_transform(documents)

# Then, apply TfidfTransformer to the term-document matrix
vc = vector.fit_transform(document_term_matrix)


In [43]:
vc.toarray()

array([[0.        , 0.        , 0.        , 0.62276601, 0.        ,
        0.62276601, 0.        , 0.4736296 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.62276601,
        0.        , 0.62276601, 0.4736296 , 0.        ],
       [0.5       , 0.5       , 0.5       , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       ]])

In [56]:
queary="you are from chaina"

In [57]:
# Transform the query using the fitted CountVectorizer
query_term_matrix = count_vectorizer.transform([queary])

# Transform the query's count vector into its TF-IDF representation
query_vc = vector.transform(query_term_matrix)



In [58]:
docs=cosine_similarity(query_vc.toarray(),vc.toarray())

In [59]:
docs

array([[0.        , 0.        , 0.70710678]])

In [66]:
vals=np.argmax(docs)

In [69]:
documents[vals]

'you am from india'

In [76]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [154]:
geminin_api_key= ""

In [83]:
llm=ChatGoogleGenerativeAI(temperature=1,model="gemini-2.0-flash",api_key=geminin_api_key)

In [85]:
llm.invoke("hi").content

'Hi there! How can I help you today?'

In [86]:
from langchain_community.document_loaders import PyPDFLoader

In [87]:
data=PyPDFLoader("/content/OfferLetter.pdf")

In [91]:
documents=data.load()

In [92]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [93]:
splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)

In [95]:
chunkDouments=splitter.split_documents(documents)

In [97]:
chunkDouments[1]

Document(metadata={'producer': 'PD4ML. HTML to PDF Converter for Java (3109fx4)', 'creator': 'PD4ML. HTML to PDF Converter for Java (3109fx4)', 'creationdate': '2025-06-13T07:24:10+00:00', 'author': 'SuccessFactors Performance Manager', 'source': '/content/OfferLetter.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='www.capgemini.com/in-en\n\xa0\nEMPLOYMENT OFFER LETTER\n\xa0\nCapgemini Ref: 2580538/201435,\n06/13/2025,\nSompalle Narendra.\n4-10, Kanganellore Mangalam\nChittoor, Andhra Pradesh\nIndia.\nConfidential\nDear Sompalle Narendra,')

In [98]:
from langchain_community.vectorstores import Chroma

In [102]:
db=Chroma

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [103]:
from langchain_google_genai.embeddings import Embeddings

In [120]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embedding_text=GoogleGenerativeAIEmbeddings(google_api_key=geminin_api_key,model="models/embedding-001")

In [125]:
db=Chroma.from_documents(chunkDouments,embedding_text)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [135]:
vector_retriver=db.as_retriever()

In [128]:
db

In [132]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [133]:
bm_retriver=BM25Retriever.from_documents(chunkDouments)

In [134]:
bm_retriver

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7b98daaed450>)

In [137]:
ensemble_retriver=EnsembleRetriever(retrievers=[vector_retriver,bm_retriver],weights=[0.3,0.7])

In [138]:
# hybride_score=(1-alpha)*keyword_search+alpha * vector_search


In [139]:
# ensemble_retriver.invoke

In [140]:
from langchain.chains import RetrievalQA

In [152]:
chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=ensemble_retriver)

In [153]:
chain.invoke("what is monthly salary to narendra")

{'query': 'what is monthly salary to narendra',
 'result': "Narendra's gross monthly salary is ₹116,598.00."}

In [148]:
chain2=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=vector_retriver)

In [151]:
chain2.invoke("what is monthly salary to narendra")

{'query': 'what is monthly salary to narendra',
 'result': "Based on the provided payslip information, Narendra's gross monthly salary is ₹116,598.00."}